In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import keras_tuner as kt
import shutil

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses, optimizers, regularizers, metrics
from tensorflow.keras.models import Model

2025-04-05 02:23:11.537691: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743812591.551743  685760 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743812591.556233  685760 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1743812591.567192  685760 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1743812591.567208  685760 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1743812591.567209  685760 computation_placer.cc:177] computation placer alr

In [2]:
dataframe = pd.read_csv('http://storage.googleapis.com/download.tensorflow.org/data/ecg.csv', header=None)
raw_data = dataframe.values
dataframe.head()

,0,1,2,3,4,5,6,7,8,9,...,131,132,133,134,135,136,137,138,139,140
0,-0.112522,-2.827204,-3.773897,-4.349751,-4.376041,-3.474986,-2.181408,-1.818286,-1.250522,-0.477492,...,0.792168,0.933541,0.796958,0.578621,0.257740,0.228077,0.123431,0.925286,0.193137,1.0
1,-1.100878,-3.996840,-4.285843,-4.506579,-4.022377,-3.234368,-1.566126,-0.992258,-0.754680,0.042321,...,0.538356,0.656881,0.787490,0.724046,0.555784,0.476333,0.773820,1.119621,-1.436250,1.0
2,-0.567088,-2.593450,-3.874230,-4.584095,-4.187449,-3.151462,-1.742940,-1.490659,-1.183580,-0.394229,...,0.886073,0.531452,0.311377,-0.021919,-0.713683,-0.532197,0.321097,0.904227,-0.421797,1.0
3,0.490473,-1.914407,-3.616364,-4.318823,-4.268016,-3.881110,-2.993280,-1.671131,-1.333884,-0.965629,...,0.350816,0.499111,0.600345,0.842069,0.952074,0.990133,1.086798,1.403011,-0.383564,1.0
4,0.800232,-0.874252,-2.384761,-3.973292,-4.338224,-3.802422,-2.534510,-1.783423,-1.594450,-0.753199,...,1.148884,0.958434,1.059025,1.371682,1.277392,0.960304,0.971020,1.614392,1.421456,1.0


In [3]:
# The last element contains the labels
labels = raw_data[:, -1]

# The other data points are the electrocadriogram data
data = raw_data[:, 0:-1]

train_data, test_data, train_labels, test_labels = train_test_split(
    data, labels, test_size=0.2, random_state=21
)

In [4]:
min_val = tf.reduce_min(train_data)
max_val = tf.reduce_max(train_data)

train_data = (train_data - min_val) / (max_val - min_val)
test_data = (test_data - min_val) / (max_val - min_val)

train_data = tf.cast(train_data, tf.float32)
test_data = tf.cast(test_data, tf.float32)

W0000 00:00:1743812594.078563  685760 gpu_device.cc:2341] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [5]:
train_labels = train_labels.astype(bool)
test_labels = test_labels.astype(bool)

normal_train_data = train_data[train_labels]
normal_test_data = test_data[test_labels]

anomalous_train_data = train_data[~train_labels]
anomalous_test_data = test_data[~test_labels]

In [6]:
def regulizer_builder(regularizer_type, reg_strength):
    if regularizer_type == 'l1' or regularizer_type == 'L1':
        kernel_reg = regularizers.L1(reg_strength)
    elif regularizer_type == 'l2' or regularizer_type == 'L2':
        kernel_reg = regularizers.L2(reg_strength)
    else:
        kernel_reg = None
        
    return kernel_reg

def model_builder(hp):

    hp_units_layer_1 = hp.Int('units_layer_1', min_value=64, max_value=128, step=32)
    hp_units_layer_2 = hp.Int('units_layer_2', min_value=32, max_value=64, step=32)
    hp_units_layer_4 = hp.Int('units_layer_4', min_value=32, max_value=64, step=32)
    hp_units_layer_5 = hp.Int('units_layer_5', min_value=64, max_value=128, step=32)
    
    hp_activation_layer_1 = hp.Choice("activation_layer_1", values=["relu","sigmoid"])
    hp_activation_layer_2 = hp.Choice("activation_layer_2", values=["relu","sigmoid"])
    hp_activation_layer_3 = hp.Choice("activation_layer_3", values=["relu","sigmoid"])
    hp_activation_layer_4 = hp.Choice("activation_layer_4", values=["relu","sigmoid"])
    hp_activation_layer_5 = hp.Choice("activation_layer_5", values=["relu","sigmoid"])

    hp_regulizer_type_layer_1 = hp.Choice("regulizer_type_layer_1", values=["None","L1","L2"])
    hp_regulizer_type_layer_2 = hp.Choice("regulizer_type_layer_2", values=["None","L1","L2"])
    hp_regulizer_type_layer_3 = hp.Choice("regulizer_type_layer_3", values=["None","L1","L2"])
    hp_regulizer_type_layer_4 = hp.Choice("regulizer_type_layer_4", values=["None","L1","L2"])
    hp_regulizer_type_layer_5 = hp.Choice("regulizer_type_layer_5", values=["None","L1","L2"])

    hp_regulizer_strength_layer_1 = hp.Choice('regulizer_strength_layer_1', values=[1e-3, 5e-3, 1e-4, 5e-4, 1e-5, 5e-5,])
    hp_regulizer_strength_layer_2 = hp.Choice('regulizer_strength_layer_2', values=[1e-3, 5e-3, 1e-4, 5e-4, 1e-5, 5e-5,])
    hp_regulizer_strength_layer_3 = hp.Choice('regulizer_strength_layer_3', values=[1e-3, 5e-3, 1e-4, 5e-4, 1e-5, 5e-5,])
    hp_regulizer_strength_layer_4 = hp.Choice('regulizer_strength_layer_4', values=[1e-3, 5e-3, 1e-4, 5e-4, 1e-5, 5e-5,])
    hp_regulizer_strength_layer_5 = hp.Choice('regulizer_strength_layer_5', values=[1e-3, 5e-3, 1e-4, 5e-4, 1e-5, 5e-5,])
    
    model = tf.keras.Sequential()
    model.add(layers.Input(shape=(140,)))
    model.add(layers.Dense(units=hp_units_layer_1, 
                     activation=hp_activation_layer_1, 
                     kernel_regularizer=regulizer_builder(hp_regulizer_type_layer_1, 
                                                          hp_regulizer_strength_layer_1)))
    model.add(layers.Dense(units=hp_units_layer_2, 
                     activation=hp_activation_layer_2, 
                     kernel_regularizer=regulizer_builder(hp_regulizer_type_layer_2, 
                                                          hp_regulizer_strength_layer_2)))
    model.add(layers.Dense(units=8, 
                           activation=hp_activation_layer_3, 
                           kernel_regularizer=regulizer_builder(hp_regulizer_type_layer_3, 
                                                                hp_regulizer_strength_layer_3)))
    model.add(layers.Dense(units=hp_units_layer_4, 
                           activation=hp_activation_layer_4, 
                           kernel_regularizer=regulizer_builder(hp_regulizer_type_layer_4, 
                                                                hp_regulizer_strength_layer_4)))
    model.add(layers.Dense(units=hp_units_layer_5, 
                     activation=hp_activation_layer_5, 
                     kernel_regularizer=regulizer_builder(hp_regulizer_type_layer_5, 
                                                          hp_regulizer_strength_layer_5)))
    model.add(layers.Dense(units=140, activation="sigmoid"))

    hp_learning_rate = hp.Choice('learning_rate', values=[1e-3, 5e-3, 1e-4, 5e-4, 1e-5, 5e-5,])
    
    model.compile(optimizer=optimizers.Adam(learning_rate=hp_learning_rate), 
                  loss=losses.MeanAbsoluteError(), 
                  metrics=[metrics.MeanAbsoluteError()])
    return model

In [7]:
%%time

shutil.rmtree('my_dir', ignore_errors=True)

tuner = kt.Hyperband(
    model_builder,
    objective="val_mean_absolute_error",
    max_epochs=10,
    directory='my_dir',
    project_name='ECG-EKG'
)

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_mean_absolute_error', patience=2)

tuner.search(normal_train_data, normal_train_data, 
             epochs=40, 
             batch_size=512, 
             validation_data=(test_data, test_data), 
             shuffle=True,
             callbacks=[stop_early],
             verbose=1)

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

Trial 30 Complete [00h 00m 03s]
val_mean_absolute_error: 0.034431710839271545

Best val_mean_absolute_error So Far: 0.03439076989889145
Total elapsed time: 00h 01m 04s
CPU times: user 1min 2s, sys: 7.05 s, total: 1min 9s
Wall time: 1min 4s


In [8]:
hypermodel = tuner.hypermodel.build(best_hps)
hypermodel.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 96)             │        13,536 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 32)             │         3,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 8)              │           264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 140)            │         9,100 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 28,404 (110.95 KB)

 Trainable params: 28,404 (110.95 KB)

 Non-trainable params: 0 (0.00 B)

In [9]:
%%time

history = hypermodel.fit(normal_train_data, normal_train_data,
        epochs=40,
        batch_size=512,
        validation_data=(test_data, test_data),
        shuffle=True,
        verbose=1)

Epoch 1/40
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - loss: 1.6605 - mean_absolute_error: 0.0795 - val_loss: 1.3564 - val_mean_absolute_error: 0.0479
Epoch 2/40
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 1.2735 - mean_absolute_error: 0.0355 - val_loss: 1.0439 - val_mean_absolute_error: 0.0407
Epoch 3/40
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.9703 - mean_absolute_error: 0.0279 - val_loss: 0.7776 - val_mean_absolute_error: 0.0366
Epoch 4/40
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.7136 - mean_absolute_error: 0.0241 - val_loss: 0.5570 - val_mean_absolute_error: 0.0364
Epoch 5/40
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.5022 - mean_absolute_error: 0.0230 - val_loss: 0.3809 - val_mean_absolute_error: 0.0355
Epoch 6/40
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.3355 - mean_absolute_error: 0.0217 - val_loss: 0.2513 - val_mean_absolute_error: 0.0350
Epoch 7/40
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.2154 - mean_absolute_error: 0.0215 - val_loss: 0.1600 - val

In [10]:
def predict(model, data, threshold):
    reconstructions = model(data)
    loss = tf.keras.losses.mae(reconstructions, data)
    return tf.math.less(loss, threshold)

def print_stats(predictions, labels):
    print("Accuracy = {}".format(accuracy_score(labels, predictions)))
    print("Precision = {}".format(precision_score(labels, predictions)))
    print("Recall = {}".format(recall_score(labels, predictions)))

In [11]:
reconstructions = hypermodel.predict(normal_train_data)
train_loss = tf.keras.losses.mae(reconstructions, normal_train_data)

74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  


In [12]:
threshold = np.mean(train_loss) + np.std(train_loss)
print("Threshold: ", threshold)

Threshold:  0.03335403


In [13]:
preds = predict(hypermodel, test_data, threshold)
print_stats(preds, test_labels)

Accuracy = 0.945
Precision = 0.9922027290448343
Recall = 0.9089285714285714
